## Task 1

In [1]:
import numpy as np

In [2]:
def toyrnd(n):
    vals = [0,1]
    # There must be some relation between x1 and x2 here
    x1 = np.random.choice(vals, size=n, p=[0.7, 0.3])
    x2 = np.random.choice(vals, size=n, p=[0.6, 0.4])
    return np.array([x1, x2])

In [3]:
n = 10000
X = toyrnd(n)
print(X.shape)

(2, 10000)


In [4]:
mu = np.sum(X, axis=1)/n
mu = np.reshape(mu, (2,1))
X_bar = X-mu
sigma = X_bar.dot(np.transpose(X_bar))/n
print(sigma)

[[0.21278959 0.00140414]
 [0.00140414 0.23930844]]


## Task 2

In [5]:
import imageio
I1 = imageio.imread("data/mnist/d0/d0_0024.png").astype(np.float64)/255
I2 = imageio.imread("data/mnist/d5/d5_0024.png").astype(np.float64)/255

In [6]:
diff = I1-I2
import time
start = time.time()
first = np.linalg.norm(diff, 'fro')
after_first = time.time()
second = np.sqrt(np.trace(np.transpose(diff).dot(diff)))
after_second = time.time()
third = np.sqrt(np.sum(np.square(diff)))
end = time.time()
print(first, " in ", after_first - start)
print(second, " in ", after_second - after_first)
print(third, " in ", end - after_second)

9.524661125214534  in  0.00012373924255371094
9.524661125214536  in  0.0003275871276855469
9.524661125214534  in  0.0002105236053466797


In [7]:
n_categories = 10
n_train = 4000
n_validation = 100
# n_test = 400
sample_size = 28 ** 2

X_train = np.zeros((n_categories * n_train, sample_size))
Y_train = np.zeros(n_categories * n_train, np.int8)
X_validation = np.zeros((n_categories * n_validation, sample_size))
Y_validation = np.zeros(n_categories * n_validation, np.int8)
# X_test = np.zeros((n_categories * n_test, sample_size))
# Y_test = np.zeros(n_categories * n_test, np.int8)

for i in range(n_categories):
    cat_str = "d" + str(i)
    pre_str = "data/mnist/" + cat_str + "/" + cat_str + "_"
    
    for j in np.arange(1,n_train+1):
        X_train[i*n_train+j-1] = imageio.imread(pre_str + str(j).zfill(4) + ".png").astype(np.float64).ravel()/255
    Y_train[i*n_train : (i+1)*n_train] = i
        
    for j in np.arange(1+n_train,1+n_train+n_validation):
        X_validation[i*n_validation+j-1-n_train] = imageio.imread(pre_str + str(j).zfill(4) + ".png").astype(np.float64).ravel()/255
    Y_validation[i*n_validation : (i+1)*n_validation] = i
        
#     for j in np.arange(1+n_train+n_validation,1+n_train+n_validation+n_test):
#         X_test[i*n_test+j-1-n_train-n_validation] = imageio.imread(pre_str + str(j).zfill(4) + ".png").astype(np.float64).ravel()/255
#     Y_test[i*n_test : (i+1)*n_test] = i

print(X_train.shape, Y_train.shape)
print(X_validation.shape, Y_validation.shape)
# print(X_test.shape, Y_test.shape)

(40000, 784) (40000,)
(1000, 784) (1000,)


In [16]:
k = 20
NN_validation = np.zeros((n_categories * n_validation, k), np.uint16)
d_validation = np.zeros((n_categories * n_validation, k))
Y_kNN_validation = np.zeros(n_categories * n_validation, np.int8)

for i in range(n_categories * n_validation):
    distances = np.linalg.norm(X_train - X_validation[i], ord=2, axis=1)
    NN_validation[i] = np.argpartition(distances, k)[:k]
    d_validation[i] = distances[NN_validation[i]]
    nearest_labels = Y_train[NN_validation[i]]
    Y_kNN_validation[i] = np.bincount(nearest_labels).argmax()

In [17]:
accuracy = (Y_kNN_validation == Y_validation).sum() / (n_categories * n_validation)
print(accuracy)
accuracies = np.zeros(n_categories)
for i in range(n_categories):
    accuracies[i] = (Y_kNN_validation[i*n_validation:(i+1)*n_validation] == Y_validation[i*n_validation:(i+1)*n_validation]).sum() / n_validation
print(accuracies)

0.956
[1.   1.   0.89 0.89 0.97 0.98 0.99 0.98 0.86 1.  ]


In [18]:
Y_kNN_validation_w = np.zeros_like(Y_kNN_validation)
d_by_cat = np.zeros(n_categories)

for i in range(n_categories * n_validation):
    Z = (1/d_validation[i]).sum()
    nearest_labels = Y_train[NN_validation[i]]
    for j in range(n_categories):
        d_by_cat[j] = (1/Z) * d_validation[i][nearest_labels == j].sum()
    Y_kNN_validation_w[i] = d_by_cat.argmax()

In [19]:
accuracy_w = (Y_kNN_validation_w == Y_validation).sum() / (n_categories * n_validation)
print(accuracy_w)
accuracies_w = np.zeros(n_categories)
for i in range(n_categories):
    accuracies_w[i] = (Y_kNN_validation_w[i*n_validation:(i+1)*n_validation] == Y_validation[i*n_validation:(i+1)*n_validation]).sum() / n_validation
print(accuracies_w)

0.956
[1.   1.   0.9  0.89 0.97 0.97 0.99 0.98 0.86 1.  ]


In [20]:
X = imageio.imread("data/mnist/d0/d0_5000.png").astype(np.float64).ravel()/255
d_X = np.linalg.norm(X_train - X, ord=2, axis=1)
NN = np.argpartition(d_X, k)[:k]
d_X_k = d_X[NN]
NN_labels = Y_train[NN]
Y = np.bincount(NN_labels).argmax()
print(Y)

0
